# Clasificador de datos que permita reconocer la marcha - Hold Out

In [1]:
import import_ipynb
from functions import *
from ciervo.plots import emg_plot

from ciervo.io import load_data
from ciervo.models import label_data, train_test_split
from tqdm import tqdm

importing Jupyter notebook from functions.ipynb
fatal: destination path 'balu3' already exists and is not an empty directory.
Processing ./balu3
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for balu3: filename=balu3-1.0-py3-none-any.whl size=43718 sha256=4dc74179b5fa99a607fbfeb010e26772ce7f3ad8156a33b4ec504d0add5be27f
  Stored in directory: /tmp/pip-ephem-wheel-cache-_nwb30oc/wheels/2e/72/90/0b6128ea07a8a26c59ffe83a05ebdf4c86dcf5b9aeefc3d561
Successfully built balu3
  Attempting uninstall: balu3
    Found existing installation: balu3 1.0
    Uninstalling balu3-1.0:
      Successfully uninstalled balu3-1.0


## 1. Data Loading

In [2]:
data_files = load_data('data/marcha_larga')

Total time: 2.29 hours


## 2. Feature Extraction 

In [3]:
#Funcion extraccion caracteristicas
def extract_feature(data, divide=3):
    # data : (T, 4) 
    # T numero de muestras, 4 canales de EMG
    # C numero de indices de canales a usar
    # divide: divide la señal en partes iguales
    _, C = data.shape
    result = []
    feature_names = []

    for c in range(C):
        signal0 = data[:, c]

        # Full wave rectification
        rectified_signal = np.abs(signal0)

        #envolvente
        env = np.abs(signal.hilbert(data[:, c]))

        #RMS
        rms = np.sqrt(np.mean(rectified_signal**2))
        result.append(rms)
        feature_names.append(f"rms_channel_{c}")

        #Varianza
        var = np.var(rectified_signal)
        result.append(var)
        feature_names.append(f"var_channel_{c}")

        #kurtosis
        kurt = scipy.stats.kurtosis(rectified_signal)
        result.append(kurt)
        feature_names.append(f"kurt_channel_{c}")

        #skewness
        skew = scipy.stats.skew(rectified_signal)
        result.append(skew)
        feature_names.append(f"skew_channel_{c}")

        #zero crossing
        zc = ((signal0[:-1] * signal0[1:]) < 0).sum()
        result.append(zc)
        feature_names.append(f"zc_channel_{c}")

        #Frecuencias
        freqs, power_spectrum = scipy.signal.welch(signal0, fs=250, nperseg=32)
        median_freq = freqs[np.where(np.cumsum(power_spectrum) >= np.sum(power_spectrum) / 2)[0][0]]
        mean_freq = np.sum(freqs * power_spectrum) / np.sum(power_spectrum)
        peak_freq = freqs[np.argmax(power_spectrum)]

        result.extend([median_freq, mean_freq, peak_freq])
        feature_names.extend([f"median_freq_channel_{c}", f"mean_freq_channel_{c}", f"peak_freq_channel_{c}"])


        #SEGMENTOS
        for i in range(divide):
            start = int(i*len(data)/divide)
            end = int((i+1)*len(data)/divide)

            segment_env = env[start:end]
            mean_env = segment_env.mean()
            std_env = segment_env.std()
            max_env = segment_env.max()
            min_env = segment_env.min()

            result.extend([mean_env, std_env, max_env, min_env])
            feature_names.extend([f"mean_env_segment_{i}channel{c}", f"std_env_segment_{i}channel{c}",
                                  f"max_env_segment_{i}channel{c}", f"min_env_segment_{i}channel{c}"])

    result = np.array(result)
    return result, feature_names

def label_data_and_features(data, divide=3):
    features =[]
    for d in tqdm(data):
      f,_ = extract_feature(d, divide)
      features.append(f)
    features = np.array(features) # (1000, features)
    return features

## 3. Training and testing subsets

## 4. Classification

In [4]:
classifiers = create_classifiers()
fases_to_test = [4, 8, 16]  # Lista de fases a probar

## 5. Evaluation

### SOLO Clean y normalizar

In [5]:
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                      columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                      window_size=125,test_size=0.2,
                                                                      overlap=0,
                                                                      random_state=42)
    
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    
    print(f"Train data: Extracted {train_data.shape[0]} samples with {train_data.shape[1]} features each.")
    print(f"Train labels shape: {train_labels.shape}")
    print("-----------------------------------------------------------------------")
    print(f"Test data: Extracted {test_data.shape[0]} samples with {test_data.shape[1]} features each.")
    print(f"Test labels shape: {test_labels.shape}")
    print("-----------------------------------------------------------------------")
    total_accuracies = evaluate_model_cleannormalize(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

TESTING WITH 4 PHASES


100%|██████████| 2437/2437 [00:05<00:00, 457.90it/s]


Train data: Extracted 9763 samples with 60 features each.
Train labels shape: (9763,)
-----------------------------------------------------------------------
Test data: Extracted 2437 samples with 60 features each.
Test labels shape: (2437,)
-----------------------------------------------------------------------

Accuracy Total por Clasificador:
knn                               : [0.5457529749692245]
knn-3                             : [0.5884283955683217]
knn-5                             : [0.6093557652851868]
knn-8                             : [0.6278210915059499]
knn-9                             : [0.6253590480098482]
knn-10                            : [0.6339762002462044]
knn-15                            : [0.6380796060730406]
knn-20                            : [0.6380796060730406]
mlp                               : [0.62330734509643]
mlp layers 2                      : [0.6758309396799344]
svm lineal 1                      : [0.5785802215839146]
svm lineal 2               

100%|██████████| 2437/2437 [00:05<00:00, 462.10it/s]


Train data: Extracted 9763 samples with 60 features each.
Train labels shape: (9763,)
-----------------------------------------------------------------------
Test data: Extracted 2437 samples with 60 features each.
Test labels shape: (2437,)
-----------------------------------------------------------------------

Accuracy Total por Clasificador:
knn                               : [0.3693065244152647]
knn-3                             : [0.3894132129667624]
knn-5                             : [0.4193680755026672]
knn-8                             : [0.43372999589659417]
knn-9                             : [0.44234714813295034]
knn-10                            : [0.44686089454247024]
knn-15                            : [0.44809191629052114]
knn-20                            : [0.44809191629052114]
mlp                               : [0.4267542059909725]
mlp layers 2                      : [0.5145670906852687]
svm lineal 1                      : [0.3881821912187115]
svm lineal 2        

100%|██████████| 2437/2437 [00:05<00:00, 462.54it/s]


Train data: Extracted 9763 samples with 60 features each.
Train labels shape: (9763,)
-----------------------------------------------------------------------
Test data: Extracted 2437 samples with 60 features each.
Test labels shape: (2437,)
-----------------------------------------------------------------------

Accuracy Total por Clasificador:
knn                               : [0.22650800164136234]
knn-3                             : [0.21460812474353713]
knn-5                             : [0.24497332786212556]
knn-8                             : [0.26179729175215427]
knn-9                             : [0.25728354534263437]
knn-10                            : [0.2622076323348379]
knn-15                            : [0.27492819039803035]
knn-20                            : [0.27492819039803035]
mlp                               : [0.24784571194091096]
mlp layers 2                      : [0.30447271235125156]
svm lineal 1                      : [0.16988100123102176]
svm lineal 2   

### SFS 

In [6]:
print(f'TESTING WITH SFS 5')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_sfs5(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

print(f'TESTING WITH SFS 10')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_sfs10(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

print(f'TESTING WITH SFS 15')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_sfs15(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

    print(f'TESTING WITH SFS 20')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_sfs20(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

print(f'TESTING WITH SFS 25')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_sfs25(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

TESTING WITH SFS 5
TESTING WITH 4 PHASES


100%|██████████| 2437/2437 [00:05<00:00, 466.07it/s]


[42 22  6 25 48]

Accuracy Total por Clasificador:
knn                               : [0.45260566270004104]
knn-3                             : [0.48256052523594584]
knn-5                             : [0.5063602790315962]
knn-8                             : [0.5145670906852687]
knn-9                             : [0.5215428805908905]
knn-10                            : [0.5240049240869922]
knn-15                            : [0.5453426343865408]
knn-20                            : [0.5453426343865408]
mlp                               : [0.45711940910956095]
mlp layers 2                      : [0.49733278621255644]
svm lineal 1                      : [0.4521953221173574]
svm lineal 2                      : [0.4587607714402954]
svm polinomial                    : [0.24784571194091096]
svm rbf 1                         : [0.3984407057858022]
svm rbf 2                         : [0.4919983586376693]
svm rbf 3                         : [0.5449322938038572]
svm rbf gamma auto              

100%|██████████| 2437/2437 [00:05<00:00, 469.08it/s]


[42 22  6 21 25]

Accuracy Total por Clasificador:
knn                               : [0.2141977841608535]
knn-3                             : [0.2306114074681986]
knn-5                             : [0.2322527697989331]
knn-8                             : [0.24620434961017645]
knn-9                             : [0.25851456709068527]
knn-10                            : [0.26056627000410343]
knn-15                            : [0.26384899466557243]
knn-20                            : [0.26384899466557243]
mlp                               : [0.212556421830119]
mlp layers 2                      : [0.19450143619203938]
svm lineal 1                      : [0.212556421830119]
svm lineal 2                      : [0.2232252769798933]
svm polinomial                    : [0.12679524004924086]
svm rbf 1                         : [0.1690603200656545]
svm rbf 2                         : [0.2425112843660238]
svm rbf 3                         : [0.2901107919573246]
svm rbf gamma auto              

...

[42 22  6 25 48 39 47 23 16 40 29 41  2  3 19 20  5 55 51  8 21 56 26 35
 36]

Accuracy Total por Clasificador:
knn                               : [0.5628840639082343]
knn-3                             : [0.5780417861532159]
knn-5                             : [0.596886521917247]
knn-8                             : [0.6058992216304793]
knn-9                             : [0.6145022531749283]
knn-10                            : [0.6185989348627612]
knn-15                            : [0.6173699303564113]
knn-20                            : [0.6173699303564113]
mlp                               : [0.5981155264235969]
mlp layers 2                      : [0.6505530520278574]
svm lineal 1                      : [0.5591970503891848]
svm lineal 2                      : [0.5817287996722654]
svm polinomial                    : [0.4027038099139697]
svm rbf 1                         : [0.5145432199918066]
svm rbf 2                         : [0.6173699303564113]
svm rbf 3                         

100%|██████████| 2441/2441 [00:05<00:00, 467.14it/s]


[42 22  6 21 25 47 55 51 33 36 29 40 39 41 20 23 45  2  3 16  7  8 27 56
 48]

Accuracy Total por Clasificador:
knn                               : [0.4018844735764031]
knn-3                             : [0.4158131913150348]
knn-5                             : [0.4326095862351495]
knn-8                             : [0.44285129045473165]
knn-9                             : [0.44285129045473165]
knn-10                            : [0.44612863580499795]
knn-15                            : [0.46292503072511265]
knn-20                            : [0.46292503072511265]
mlp                               : [0.397378123719787]
mlp layers 2                      : [0.4342482589102827]
svm lineal 1                      : [0.3678820155673904]
svm lineal 2                      : [0.39164276935682096]
svm polinomial                    : [0.2183531339614912]
svm rbf 1                         : [0.2580909463334699]
svm rbf 2                         : [0.4477673084801311]
svm rbf 3                   

100%|██████████| 2441/2441 [00:05<00:00, 454.82it/s]


[42 22  6 21 25 47 55 51 28 33 36 40 39 20 41 23 35  3  2 56 45 19  8 52
 48]

Accuracy Total por Clasificador:
knn                               : [0.2666939778779189]
knn-3                             : [0.2380172060630889]
knn-5                             : [0.2666939778779189]
knn-8                             : [0.28553871364195005]
knn-9                             : [0.2949610815239656]
knn-10                            : [0.2892257271609996]
knn-15                            : [0.30110610405571486]
knn-20                            : [0.30110610405571486]
mlp                               : [0.19500204834084392]
mlp layers 2                      : [0.20483408439164277]
svm lineal 1                      : [0.13191315034821793]
svm lineal 2                      : [0.1753379762392462]
svm polinomial                    : [0.07619827939369112]
svm rbf 1                         : [0.09135600163867268]
svm rbf 2                         : [0.23596886521917246]
svm rbf 3               

### PCA

In [7]:
print(f'TESTING WITH PCA 10')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_pca10(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

print(f'TESTING WITH PCA 20')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_pca20(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

print(f'TESTING WITH PCA 25')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_pca25(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

TESTING WITH PCA 10
TESTING WITH 4 PHASES


100%|██████████| 2441/2441 [00:05<00:00, 462.66it/s]



Accuracy Total por Clasificador:
knn                               : [0.533797623924621]
knn-3                             : [0.5579680458828349]
knn-5                             : [0.5903318312167145]
knn-8                             : [0.5936091765669808]
knn-9                             : [0.5956575174108971]
knn-10                            : [0.5936091765669808]
knn-15                            : [0.6013928717738631]
knn-20                            : [0.6013928717738631]
mlp                               : [0.5083981974600573]
mlp layers 2                      : [0.594018844735764]
svm lineal 1                      : [0.47726341663252764]
svm lineal 2                      : [0.49446947972142563]
svm polinomial                    : [0.2572716099959033]
svm rbf 1                         : [0.4752150757886112]
svm rbf 2                         : [0.5473166734944694]
svm rbf 3                         : [0.6349856616140925]
svm rbf gamma auto                : [0.503891847603441

100%|██████████| 2441/2441 [00:05<00:00, 469.73it/s]



Accuracy Total por Clasificador:
knn                               : [0.3637853338795576]
knn-3                             : [0.362965997541991]
knn-5                             : [0.39123310118803767]
knn-8                             : [0.39573945104465386]
knn-9                             : [0.4006554690700533]
knn-10                            : [0.40024580090127]
knn-15                            : [0.420319541171651]
knn-20                            : [0.420319541171651]
mlp                               : [0.28390004096681687]
mlp layers 2                      : [0.31544448996312985]
svm lineal 1                      : [0.24293322408848833]
svm lineal 2                      : [0.29782875870544856]
svm polinomial                    : [0.12576812781646865]
svm rbf 1                         : [0.20032773453502664]
svm rbf 2                         : [0.3510856206472757]
svm rbf 3                         : [0.48340843916427695]
svm rbf gamma auto                : [0.30479311757

...


Accuracy Total por Clasificador:
knn                               : [0.5030725112658746]
knn-3                             : [0.5247849242113888]
knn-5                             : [0.556739041376485]
knn-8                             : [0.5665710774272839]
knn-9                             : [0.5731257681278165]
knn-10                            : [0.5784514543219992]
knn-15                            : [0.5936091765669808]
knn-20                            : [0.5936091765669808]
mlp                               : [0.5620647275706677]
mlp layers 2                      : [0.6267922982384269]
svm lineal 1                      : [0.5436296599754199]
svm lineal 2                      : [0.5546907005325686]
svm polinomial                    : [0.5436296599754199]
svm rbf 1                         : [0.5362556329373208]
svm rbf 2                         : [0.6034412126177796]
svm rbf 3                         : [0.6820975010241704]
svm rbf gamma auto                : [0.5444489963129865

100%|██████████| 2441/2441 [00:05<00:00, 488.10it/s]



Accuracy Total por Clasificador:
knn                               : [0.3248668578451454]
knn-3                             : [0.334289225727161]
knn-5                             : [0.3617369930356411]
knn-8                             : [0.3793527242933224]
knn-9                             : [0.3854977468250717]
knn-10                            : [0.39123310118803767]
knn-15                            : [0.3998361327324867]
knn-20                            : [0.3998361327324867]
mlp                               : [0.34985661614092584]
mlp layers 2                      : [0.3064317902498976]
svm lineal 1                      : [0.34289225727161]
svm lineal 2                      : [0.3674723473986071]
svm polinomial                    : [0.32158951249487916]
svm rbf 1                         : [0.24006554690700532]
svm rbf 2                         : [0.414584186808685]
svm rbf 3                         : [0.5243752560426055]
svm rbf gamma auto                : [0.322818517001229

100%|██████████| 2441/2441 [00:05<00:00, 485.78it/s]



Accuracy Total por Clasificador:
knn                               : [0.1904956984842278]
knn-3                             : [0.18639901679639492]
knn-5                             : [0.19909873002867678]
knn-8                             : [0.22122081114297418]
knn-9                             : [0.23023351085620647]
knn-10                            : [0.23433019254403933]
knn-15                            : [0.23637853338795575]
knn-20                            : [0.23637853338795575]
mlp                               : [0.15936091765669808]
mlp layers 2                      : [0.10323637853338796]
svm lineal 1                      : [0.1474805407619828]
svm lineal 2                      : [0.19500204834084392]
svm polinomial                    : [0.14379352724293323]
svm rbf 1                         : [0.07496927488734126]
svm rbf 2                         : [0.22982384268742317]
svm rbf 3                         : [0.3473986071282261]
svm rbf gamma auto                : [0.12

### ICA

In [8]:
print(f'TESTING WITH ICA 10')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_ica10(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

print(f'TESTING WITH ICA 20')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_ica20(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

print(f'TESTING WITH ICA 25')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_ica25(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

TESTING WITH ICA 10
TESTING WITH 4 PHASES


100%|██████████| 2441/2441 [00:04<00:00, 488.65it/s]



Accuracy Total por Clasificador:
knn                               : [0.5235559197050389]
knn-3                             : [0.5477263416632527]
knn-5                             : [0.5829578041786153]
knn-8                             : [0.5854158131913151]
knn-9                             : [0.5870544858664482]
knn-10                            : [0.5800901269971324]
knn-15                            : [0.5927898402294142]
knn-20                            : [0.5927898402294142]
mlp                               : [0.2466202376075379]
mlp layers 2                      : [0.2466202376075379]
svm lineal 1                      : [0.2466202376075379]
svm lineal 2                      : [0.2466202376075379]
svm polinomial                    : [0.2466202376075379]
svm rbf 1                         : [0.2466202376075379]
svm rbf 2                         : [0.2466202376075379]
svm rbf 3                         : [0.6304793117574764]
svm rbf gamma auto                : [0.246620237607537

100%|██████████| 2441/2441 [00:05<00:00, 485.22it/s]



Accuracy Total por Clasificador:
knn                               : [0.360098320360508]
knn-3                             : [0.3633756657107743]
knn-5                             : [0.39287177386317085]
knn-8                             : [0.4080294961081524]
knn-9                             : [0.4022941417451864]
knn-10                            : [0.3990167963949201]
knn-15                            : [0.42728390004096684]
knn-20                            : [0.42728390004096684]
mlp                               : [0.12453912331011881]
mlp layers 2                      : [0.12453912331011881]
svm lineal 1                      : [0.12453912331011881]
svm lineal 2                      : [0.12453912331011881]
svm polinomial                    : [0.12453912331011881]
svm rbf 1                         : [0.12453912331011881]
svm rbf 2                         : [0.12453912331011881]
svm rbf 3                         : [0.48299877099549365]
svm rbf gamma auto                : [0.12453

...


Accuracy Total por Clasificador:
knn                               : [0.5251945923801721]
knn-3                             : [0.5436296599754199]
knn-5                             : [0.5706677591151168]
knn-8                             : [0.5927898402294142]
knn-9                             : [0.5903318312167145]
knn-10                            : [0.5944285129045473]
knn-15                            : [0.6005735354362965]
knn-20                            : [0.6005735354362965]
mlp                               : [0.2519459238017206]
mlp layers 2                      : [0.2519459238017206]
svm lineal 1                      : [0.2466202376075379]
svm lineal 2                      : [0.2466202376075379]
svm polinomial                    : [0.2466202376075379]
svm rbf 1                         : [0.2466202376075379]
svm rbf 2                         : [0.2466202376075379]
svm rbf 3                         : [0.7054485866448177]
svm rbf gamma auto                : [0.246620237607537

100%|██████████| 2441/2441 [00:05<00:00, 476.50it/s]



Accuracy Total por Clasificador:
knn                               : [0.35026628430970913]
knn-3                             : [0.35436296599754197]
knn-5                             : [0.4014748054076198]
knn-8                             : [0.41171650962720197]
knn-9                             : [0.41540352314625156]
knn-10                            : [0.4190905366653011]
knn-15                            : [0.4211388775092175]
knn-20                            : [0.4211388775092175]
mlp                               : [0.12494879147890209]
mlp layers 2                      : [0.12453912331011881]
svm lineal 1                      : [0.12494879147890209]
svm lineal 2                      : [0.12494879147890209]
svm polinomial                    : [0.12494879147890209]
svm rbf 1                         : [0.12494879147890209]
svm rbf 2                         : [0.12494879147890209]
svm rbf 3                         : [0.5559197050389185]
svm rbf gamma auto                : [0.1249

100%|██████████| 2441/2441 [00:05<00:00, 460.21it/s]



Accuracy Total por Clasificador:
knn                               : [0.21466612044244163]
knn-3                             : [0.20606308889799263]
knn-5                             : [0.23351085620647274]
knn-8                             : [0.2576812781646866]
knn-9                             : [0.2585006145022532]
knn-10                            : [0.26259729619008604]
knn-15                            : [0.2695616550594019]
knn-20                            : [0.2695616550594019]
mlp                               : [0.06145022531749283]
mlp layers 2                      : [0.06145022531749283]
svm lineal 1                      : [0.06145022531749283]
svm lineal 2                      : [0.06145022531749283]
svm polinomial                    : [0.06145022531749283]
svm rbf 1                         : [0.06145022531749283]
svm rbf 2                         : [0.06145022531749283]
svm rbf 3                         : [0.3789430561245391]
svm rbf gamma auto                : [0.0614

### PLSR

In [9]:
print(f'TESTING WITH PLSR 10')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_plsr10(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

print(f'TESTING WITH PLSR 20')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_plsr20(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

print(f'TESTING WITH PLSR 25')
for num_fases in fases_to_test:
    print(f'TESTING WITH {num_fases} PHASES')
    labeled_data=label_data(data_files,num_fases=num_fases)
    train_window, train_labels, test_window, test_labels = train_test_split(labeled_data, 
                                                                          columna=["EMG_Isquio","EMG_Cuadriceps","EMG_AductorLargo"],
                                                                          window_size=125,test_size=0.2,
                                                                          overlap=0,
                                                                          random_state=42)
    train_data = label_data_and_features(train_window, divide=3)
    test_data = label_data_and_features(test_window, divide=3)
    total_accuracies = evaluate_model_plsr25(train_data, test_data, train_labels, test_labels, classifiers)
    print_accuracies(total_accuracies)

TESTING WITH PLSR 10
TESTING WITH 4 PHASES


100%|██████████| 2441/2441 [00:05<00:00, 461.76it/s]



Accuracy Total por Clasificador:
knn                               : [0.5587873822204015]
knn-3                             : [0.5837771405161819]
knn-5                             : [0.605489553461696]
knn-8                             : [0.6095862351495289]
knn-9                             : [0.6145022531749283]
knn-10                            : [0.6206472757066775]
knn-15                            : [0.6231052847193773]
knn-20                            : [0.6231052847193773]
mlp                               : [0.6468660385088079]
mlp layers 2                      : [0.6546497337156902]
svm lineal 1                      : [0.5637034002458009]
svm lineal 2                      : [0.5665710774272839]
svm polinomial                    : [0.6374436706267923]
svm rbf 1                         : [0.614092585006145]
svm rbf 2                         : [0.6394920114707088]
svm rbf 3                         : [0.642769356820975]
svm rbf gamma auto                : [0.6698074559606718]


100%|██████████| 2441/2441 [00:05<00:00, 464.29it/s]



Accuracy Total por Clasificador:
knn                               : [0.403113478082753]
knn-3                             : [0.42318721835313394]
knn-5                             : [0.4420319541171651]
knn-8                             : [0.44899631298648096]
knn-9                             : [0.45759934453092993]
knn-10                            : [0.4530929946743138]
knn-15                            : [0.46415403523146254]
knn-20                            : [0.46415403523146254]
mlp                               : [0.47931175747644406]
mlp layers 2                      : [0.5067595247849243]
svm lineal 1                      : [0.39819746005735357]
svm lineal 2                      : [0.40024580090127]
svm polinomial                    : [0.4735764031134781]
svm rbf 1                         : [0.44039328144203194]
svm rbf 2                         : [0.4789020893076608]
svm rbf 3                         : [0.48217943465792706]
svm rbf gamma auto                : [0.517410897

...


Accuracy Total por Clasificador:
knn                               : [0.5899221630479312]
knn-3                             : [0.608357230643179]
knn-5                             : [0.637034002458009]
knn-8                             : [0.6460467021712413]
knn-9                             : [0.6468660385088079]
knn-10                            : [0.6493240475215076]
knn-15                            : [0.6501433838590741]
knn-20                            : [0.6501433838590741]
mlp                               : [0.6960262187628021]
mlp layers 2                      : [0.7185579680458828]
svm lineal 1                      : [0.6128635804997952]
svm lineal 2                      : [0.614092585006145]
svm polinomial                    : [0.7034002458009013]
svm rbf 1                         : [0.6480950430151577]
svm rbf 2                         : [0.6550594018844735]
svm rbf 3                         : [0.6853748463744367]
svm rbf gamma auto                : [0.6939778779188857]


100%|██████████| 2441/2441 [00:05<00:00, 469.21it/s]



Accuracy Total por Clasificador:
knn                               : [0.4412126177795985]
knn-3                             : [0.4387546087668988]
knn-5                             : [0.4707087259319951]
knn-8                             : [0.4760344121261778]
knn-9                             : [0.48054076198279394]
knn-10                            : [0.48586644817697666]
knn-15                            : [0.4928308070462925]
knn-20                            : [0.4928308070462925]
mlp                               : [0.5518230233510856]
mlp layers 2                      : [0.560835723064318]
svm lineal 1                      : [0.46579270790659566]
svm lineal 2                      : [0.4682507169192954]
svm polinomial                    : [0.5596067185579681]
svm rbf 1                         : [0.48586644817697666]
svm rbf 2                         : [0.49897582957804176]
svm rbf 3                         : [0.5456780008193364]
svm rbf gamma auto                : [0.56452273658

100%|██████████| 2428/2428 [00:05<00:00, 475.57it/s]



Accuracy Total por Clasificador:
knn                               : [0.2701812191103789]
knn-3                             : [0.2528830313014827]
knn-5                             : [0.28953871499176276]
knn-8                             : [0.3060131795716639]
knn-9                             : [0.31466227347611203]
knn-10                            : [0.30807248764415157]
knn-15                            : [0.3245469522240527]
knn-20                            : [0.3245469522240527]
mlp                               : [0.3982701812191104]
mlp layers 2                      : [0.4258649093904448]
svm lineal 1                      : [0.313838550247117]
svm lineal 2                      : [0.3187808896210873]
svm polinomial                    : [0.4110378912685338]
svm rbf 1                         : [0.32331136738056016]
svm rbf 2                         : [0.33319604612850084]
svm rbf 3                         : [0.39373970345963755]
svm rbf gamma auto                : [0.4122734761